Lockstar - SinglePIDController - Basic Client

In [ ]:
from lockstar_client.SinglePIDClient import SinglePIDClient

In [ ]:
#connect to client
client = SinglePIDClient('192.168.88.201', 10780, 1234)

if client.register_client_id():
    print(f'Successfully initialized AWG module')

In [ ]:
client.set_output_limits(min=0, max=10)

In [ ]:
client.set_pid(p=1, i=0, d=0)

In [ ]:
client.lock()

In [ ]:
client.unlock()